In [3]:
english = ["A", "B", "C", "D", "E", "I"];
greek = ["α", "β", "γ", "δ", "ϵ", "ϕ"];
alphabets = [];

append!(alphabets, english)
append!(alphabets, greek)

12-element Vector{Any}:
 "A"
 "B"
 "C"
 "D"
 "E"
 "I"
 "α"
 "β"
 "γ"
 "δ"
 "ϵ"
 "ϕ"

In [4]:
P = Dict();
for alphabet in alphabets
    if alphabet ∈ greek
        P[alphabet] = 1 // 12;
    elseif alphabet ∈ ["A", "E", "I"]
        P[alphabet] = 1 // 9;
    else
        P[alphabet] = 1 // 18;
    end
end

In [10]:
P_2 = Dict();
for (alphabet, probability) in P
    for (alphabet_2, probability_2) in P
        P_2[alphabet * alphabet_2] = probability * probability_2;
    end
end

P_3 = Dict();
for (alphabet, probability) in P
    for (alphabet_2, probability_2) in P_2
        P_3[alphabet * alphabet_2] = probability * probability_2;
    end
end

In [5]:
function sort_prob(P_Dict)
    probabilities = [];
    alphabets = [];
    for (alphabet, probability) in P_Dict
        append!(probabilities, probability)
        append!(alphabets, alphabet)
    end

    sorted_idx = sortperm(probabilities);
    probabilities = probabilities[sorted_idx];
    alphabets = alphabets[sorted_idx];

    println("$alphabets")
    println("$probabilities")
    println("$(probabilities[sorted_idx])")

    P_sorted_dict = Dict();
    for (idx, probability) in enumerate(probabilities)
        P_sorted_dict[ string(alphabets[idx]) ] = probability
    end
    return probabilities, alphabets, P_sorted_dict
end

probabilities, alphabets, P_sorted_dict = sort_prob(P)

Any['C', 'B', 'D', 'ϵ', 'δ', 'ϕ', 'α', 'γ', 'β', 'A', 'E', 'I']
Any[1//18, 1//18, 1//18, 1//12, 1//12, 1//12, 1//12, 1//12, 1//12, 1//9, 1//9, 1//9]
Any[1//18, 1//12, 1//12, 1//18, 1//18, 1//12, 1//12, 1//9, 1//9, 1//12, 1//12, 1//9]


(Any[1//18, 1//18, 1//18, 1//12, 1//12, 1//12, 1//12, 1//12, 1//12, 1//9, 1//9, 1//9], Any['C', 'B', 'D', 'ϵ', 'δ', 'ϕ', 'α', 'γ', 'β', 'A', 'E', 'I'], Dict{Any, Any}("C" => 1//18, "ϵ" => 1//12, "δ" => 1//12, "B" => 1//18, "A" => 1//9, "ϕ" => 1//12, "D" => 1//18, "α" => 1//12, "E" => 1//9, "γ" => 1//12…))

In [6]:
mutable struct huffman_node{A, B}
    code::A
    probability::B
    left::Union{Nothing, huffman_node}
    right::Union{Nothing, huffman_node}
end
# For Huffman Leaf Construction:
huffman_node(x::A, y::B) where A where B = huffman_node{A, B}(x, y, nothing, nothing)

function construct_huffman_tree(P_sorted_dict)
    trees = [huffman_node( alphabet, probability) for (alphabet, probability) in P_sorted_dict]
    while length(trees) > 1
        sort!(trees, lt = (x, y) -> x.probability < y.probability)
        smallest = popfirst!(trees)
        next_smallest = popfirst!(trees)
        tree = huffman_node("s", smallest.probability + next_smallest.probability)
        tree.left = smallest
        tree.right = next_smallest
        push!(trees, tree )
    end
    return trees[1]
end

huffman_tree = construct_huffman_tree(P_sorted_dict)

In [13]:
function print_encoding(node, code, encoder)
    code *= "0"
    if ~(isnothing(node.left))
        print_encoding(node.left, code, encoder)
    end
    code = code[1:end-1]

    code *= "1"
    if ~(isnothing(node.right))
        print_encoding(node.right, code, encoder)
    end
    code = code[1:end-1]
    encoder[node.code] = code

end

encoder = Dict()
@time print_encoding(trees[1], "", encoder)

  0.024813 seconds (51.58 k allocations: 2.474 MiB, 99.76% compilation time)


""

In [14]:
encoder

Dict{Any, Any} with 13 entries:
  "C" => "1000"
  "ϵ" => "1011"
  "δ" => "1100"
  "B" => "1001"
  "s" => ""
  "A" => "001"
  "ϕ" => "1101"
  "D" => "1010"
  "α" => "1110"
  "E" => "010"
  "γ" => "1111"
  "I" => "011"
  "β" => "000"

In [9]:
length(encoder["A"])

3